<a href="https://colab.research.google.com/github/sunhuanhuan920/ECE1512_2023F_ProjectRepo_Zhenhuan_Sun/blob/main/Project%20B/Task_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1

The papaper I chose is "Towards Lossless Dataset Distillation via Difficulty-Aligned Trajectory Matching"

(a) Conventional dataset distillation methods become less effective when the total number of synthetic samples becomes larger. By performing experiments, this paper has given us insights about why dataset distillation become less effective when image per class (IPC) increases, by which they proposed a method that allows trajectory matching-based dataset distillation methods remain effective in both low and high IPC settings. In addition, the proposed method has managed to achieve lossless dataset distillation, allowing synthetic dataset become as effective as the original large dataset in terms of training models.

(b) Trajectory matching-based dataset distillation methods usually try to match all the update trajectories of models trained on both real and synthetic datasets across all training phases. However, the authors of this paper have found that matching late trajectories, i.e., learn "hard" patterns from original dataset, become more and more beneficial when IPC increases. While matching early trajectories, i.e., learn "easy" pattern from original dataset, is more beneficial with a low IPC and tends to be counterproductive with a high IPC. Building upon these insights, the authors propose a method that allows us to control the difficulty of the learned pattern by only matching trajectories of a specific range, based on the size of the synthetic dataset.

(c)

(d)

# 2